In [20]:
import numpy as np 
import torch 
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [5]:
class DBN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(773,100)
        self.fc2 = nn.Linear(100,100)
        self.fc3 = nn.Linear(100,100)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        return x

In [6]:
model = DBN()

In [10]:
from extractor import get_dataset
path = 'dataset/games.pgn'
num_games = 50

X, Y = get_dataset(path, num_games)

games parsed 1
games parsed 2
games parsed 3
games parsed 4
games parsed 5
games parsed 6
games parsed 7
games parsed 8
games parsed 9
games parsed 10
games parsed 11
games parsed 12
games parsed 13
games parsed 14
games parsed 15
games parsed 16
games parsed 17
games parsed 18
games parsed 19
games parsed 20
games parsed 21
games parsed 22
games parsed 23
games parsed 24
games parsed 25
games parsed 26
games parsed 27
games parsed 28
games parsed 29
games parsed 30
games parsed 31
games parsed 32
games parsed 33
games parsed 34
games parsed 35
games parsed 36
games parsed 37
games parsed 38
games parsed 39
games parsed 40
games parsed 41
games parsed 42
games parsed 43
games parsed 44
games parsed 45
games parsed 46
games parsed 47
games parsed 48
games parsed 49
games parsed 50
done parsing 50 games
games loaded = 35
encoded game 1
encoded game 2
encoded game 3
encoded game 4
encoded game 5
encoded game 6
encoded game 7
encoded game 8
encoded game 9
encoded game 10
encoded game 11
en

In [21]:
train_dataset = TensorDataset(X,Y)
train_datalaoder = DataLoader(train_dataset, batch_size=64, shuffle=True)